# Activity
path to file

```
/datasets/employee_work_records.csv
```

### Tasks

1. Display the first 10 rows of the dataset to understand its schema and contents.
2. Print the schema of the DataFrame and identify the data types of each column.
3. Count how many rows contain at least one null value.
4. Replace all null values in the hours_worked column with the average hours worked (calculated from non-null rows).
5. Replace all the null values in location as Remote
6. Drop all rows that have a null value in either salary or projects_completed.
7. Add a new column named hourly_rate which is calculated as:
  `salary / (hours_worked × 4)`
8. Group the data by department and calculate:
  - Average salary
  - Average hours_worked
  - Total projects completed
9. Group by location and count how many employees work in each location.
10. Find the maximum salary and minimum salary
11. Filter out employees who have completed more than 5 projects and worked less than 35 hours.
12. Create a temporary view named employees.
Using Spark SQL:
  - Select department-wise employee count and average salary.
  - Filter only departments where average salary is above 75,000.
  - Write a SQL query to get count of whose salary is above the average salary.
13. Create a column bonus_eligible where employees with projects_completed > 5 are marked "Yes", else "No".


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("demoDFEmployee").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/06 09:43:02 WARN Utils: Your hostname, codespaces-908b19, resolves to a loopback address: 127.0.0.1; using 10.0.1.103 instead (on interface eth0)
25/08/06 09:43:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/06 09:43:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
emp_raw=spark.read.csv("employee_work_records.csv",header=True)

In [3]:
#1
emp_raw.take(10)

[Row(id='1', name='Charlie', age='38', gender='Non-Binary', department='Finance', salary=None, projects_completed='5.0', hours_worked='46.6', location=None),
 Row(id='2', name='David', age='48', gender='Non-Binary', department='Engineering', salary='92650.0', projects_completed=None, hours_worked='40.0', location=None),
 Row(id='3', name='Frank', age='28', gender='Female', department='Marketing', salary=None, projects_completed=None, hours_worked='37.3', location='San Francisco'),
 Row(id='4', name='Bob', age='36', gender='Male', department='Sales', salary='105597.0', projects_completed='8.0', hours_worked='36.9', location=None),
 Row(id='5', name='Charlie', age='59', gender='Non-Binary', department='HR', salary=None, projects_completed=None, hours_worked='43.5', location='Chicago'),
 Row(id='6', name='Alice', age='28', gender='Non-Binary', department='Sales', salary=None, projects_completed='5.0', hours_worked='40.4', location=None),
 Row(id='7', name='Ian', age='33', gender='Male', d

In [4]:
#2

emp_raw.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- projects_completed: string (nullable = true)
 |-- hours_worked: string (nullable = true)
 |-- location: string (nullable = true)



In [5]:
#3
from pyspark.sql.types import *
from pyspark.sql.functions import *
null_count = emp_raw.filter(
    col("id").isNull() |
    col("name").isNull() |
    col("age").isNull() |
    col("gender").isNull() |
    col("department").isNull() |
    col("salary").isNull() |
    col("projects_completed").isNull() |
    col("hours_worked").isNull() |
    col("location").isNull() ).count()
print(null_count)



95


In [16]:
#4
avg_hours = emp_raw.select(avg("hours_worked")).first()[0]
emp_raw = emp_raw.fillna({"hours_worked":avg_hours})
emp_raw.show()


+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+
| id|   name|age|    gender| department|  salary|projects_completed|      hours_worked|     location|
+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+
|  4|    Bob| 36|      Male|      Sales|105597.0|               8.0|              36.9|         NULL|
|  7|    Ian| 33|      Male|Engineering| 87064.0|               1.0|              41.4|       Austin|
| 30|  David| 32|Non-Binary|      Sales| 63742.0|               4.0|39.707692307692305|         NULL|
| 32|Charlie| 47|      Male|         HR|106118.0|               4.0|              33.8|      Chicago|
| 34|    Ian| 54|    Female|    Finance| 57109.0|               8.0|              49.3|San Francisco|
| 37|  Frank| 29|      Male|  Marketing|100759.0|              10.0|39.707692307692305|San Francisco|
| 39|  Frank| 47|      Male|    Finance| 68032.0|               5.0|              

In [17]:
#5
emp_raw = new1.fillna({"location": "Remote"})
emp_raw.show()



+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+
| id|   name|age|    gender| department|  salary|projects_completed|      hours_worked|     location|
+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+
|  4|    Bob| 36|      Male|      Sales|105597.0|               8.0|              36.9|       Remote|
|  7|    Ian| 33|      Male|Engineering| 87064.0|               1.0|              41.4|       Austin|
| 30|  David| 32|Non-Binary|      Sales| 63742.0|               4.0|39.707692307692305|       Remote|
| 32|Charlie| 47|      Male|         HR|106118.0|               4.0|              33.8|      Chicago|
| 34|    Ian| 54|    Female|    Finance| 57109.0|               8.0|              49.3|San Francisco|
| 37|  Frank| 29|      Male|  Marketing|100759.0|              10.0|39.707692307692305|San Francisco|
| 39|  Frank| 47|      Male|    Finance| 68032.0|               5.0|              

In [18]:
#6
emp_raw = emp_raw.dropna(subset=["salary", "projects_completed"])
emp_raw.show()


+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+
| id|   name|age|    gender| department|  salary|projects_completed|      hours_worked|     location|
+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+
|  4|    Bob| 36|      Male|      Sales|105597.0|               8.0|              36.9|       Remote|
|  7|    Ian| 33|      Male|Engineering| 87064.0|               1.0|              41.4|       Austin|
| 30|  David| 32|Non-Binary|      Sales| 63742.0|               4.0|39.707692307692305|       Remote|
| 32|Charlie| 47|      Male|         HR|106118.0|               4.0|              33.8|      Chicago|
| 34|    Ian| 54|    Female|    Finance| 57109.0|               8.0|              49.3|San Francisco|
| 37|  Frank| 29|      Male|  Marketing|100759.0|              10.0|39.707692307692305|San Francisco|
| 39|  Frank| 47|      Male|    Finance| 68032.0|               5.0|              

In [37]:
#7
emp_raw = emp_raw.withColumn("salary", col("salary").cast("double")) \
                    .withColumn("hours_worked", col("hours_worked").cast("double")) \
                    .withColumn("age", col("age").cast("int")) \
                    .withColumn("projects_completed",col("projects_completed").cast("double")) \
                    .withColumn("id", col("id").cast("int"))
emp_raw = emp_raw.withColumn("hourly_rate", col("salary") / (col("hours_worked") * 4))
emp_raw.show()



+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+------------------+
| id|   name|age|    gender| department|  salary|projects_completed|      hours_worked|     location|       hourly_rate|
+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+------------------+
|  4|    Bob| 36|      Male|      Sales|105597.0|               8.0|              36.9|       Remote| 715.4268292682927|
|  7|    Ian| 33|      Male|Engineering| 87064.0|               1.0|              41.4|       Austin| 525.7487922705315|
| 30|  David| 32|Non-Binary|      Sales| 63742.0|               4.0|39.707692307692305|       Remote|401.32022471910113|
| 32|Charlie| 47|      Male|         HR|106118.0|               4.0|              33.8|      Chicago| 784.8964497041421|
| 34|    Ian| 54|    Female|    Finance| 57109.0|               8.0|              49.3|San Francisco| 289.5993914807302|
| 37|  Frank| 29|      Male|  Ma

In [38]:
#8
emp_raw.groupBy("department").agg(avg("salary").alias("avg_salary"),avg("hours_worked").alias("avg_hours_worked"),sum("projects_completed").alias("total_projects_completed")).show()


+-----------+-----------------+------------------+------------------------+
| department|       avg_salary|  avg_hours_worked|total_projects_completed|
+-----------+-----------------+------------------+------------------------+
|      Sales|         86622.25| 36.95384615384615|                    27.0|
|Engineering|         84625.75| 39.20192307692307|                    15.0|
|         HR|75329.28571428571| 39.51868131868132|                    24.0|
|    Finance|          74568.0|42.888461538461534|                    33.0|
|  Marketing|          80327.2| 38.76307692307692|                    25.0|
+-----------+-----------------+------------------+------------------------+



In [39]:
#9
emp_raw.groupBy("location").agg(count("id").alias("total_employees")).show()

+-------------+---------------+
|     location|total_employees|
+-------------+---------------+
|       Remote|             15|
|San Francisco|              3|
|       Austin|              3|
|      Chicago|              2|
|     New York|              1|
|       Boston|              2|
+-------------+---------------+



In [40]:
#10
emp_raw.agg(max("salary").alias("Max_salary"),min("salary").alias("Min_salary")).show()

+----------+----------+
|Max_salary|Min_salary|
+----------+----------+
|  115186.0|   40228.0|
+----------+----------+



In [41]:
#11
emp_raw.filter((col("projects_completed")>5) & (col("hours_worked")<35)).show()

+---+----+---+------+-----------+-------+------------------+------------+--------+----------------+
| id|name|age|gender| department| salary|projects_completed|hours_worked|location|     hourly_rate|
+---+----+---+------+-----------+-------+------------------+------------+--------+----------------+
| 92| Ian| 51|  Male|Engineering|86999.0|               9.0|        34.5|  Remote|630.427536231884|
+---+----+---+------+-----------+-------+------------------+------------+--------+----------------+



In [54]:
#12
emp_raw.createOrReplaceTempView("employees")
spark.sql("select department,count(id) as emp_count,avg(salary) as avg_sal from employees group by department").show()
spark.sql("select department from employees group by department having avg(salary)>75000").show()
spark.sql("select count(id) as count from employees where salary >(select avg(salary) from employees)").show()

+-----------+---------+-----------------+
| department|emp_count|          avg_sal|
+-----------+---------+-----------------+
|      Sales|        4|         86622.25|
|Engineering|        4|         84625.75|
|         HR|        7|75329.28571428571|
|    Finance|        6|          74568.0|
|  Marketing|        5|          80327.2|
+-----------+---------+-----------------+

+-----------+
| department|
+-----------+
|      Sales|
|Engineering|
|         HR|
|  Marketing|
+-----------+

+-----+
|count|
+-----+
|   14|
+-----+



In [ ]:
#13
emp_raw=emp_raw.withColumn("bonus_eligible",when(col("projects_completed")>5,"Yes").otherwise ("No"))
emp_raw.show()

+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+------------------+--------------+
| id|   name|age|    gender| department|  salary|projects_completed|      hours_worked|     location|       hourly_rate|bonus_eligible|
+---+-------+---+----------+-----------+--------+------------------+------------------+-------------+------------------+--------------+
|  4|    Bob| 36|      Male|      Sales|105597.0|               8.0|              36.9|       Remote| 715.4268292682927|           Yes|
|  7|    Ian| 33|      Male|Engineering| 87064.0|               1.0|              41.4|       Austin| 525.7487922705315|            No|
| 30|  David| 32|Non-Binary|      Sales| 63742.0|               4.0|39.707692307692305|       Remote|401.32022471910113|            No|
| 32|Charlie| 47|      Male|         HR|106118.0|               4.0|              33.8|      Chicago| 784.8964497041421|            No|
| 34|    Ian| 54|    Female|    Finance| 57109.0